In [6]:
import pandas as pd
import numpy as np

In [7]:
datafn = pd.read_csv("full_variants/caqtls.eu.lcls.benchmarking.all.tsv", sep="\t", header=0)


In [9]:
datafn["log10p"] = np.log10(datafn["obs.pval"])*-1
threshold=6

dataf1 = datafn[datafn["log10p"]>threshold]
dataf1["pos0"] = dataf1["var.pos_hg38"] - 1
peaks_var = dataf1[["var.chr", "pos0", "var.pos_hg38"]].reset_index(drop=True)
#dataf1



/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [11]:
import pybedtools
peaks_data_merged=pd.read_csv("/mnt/lab_data2/anusri/kumasaka/new_download/Kumasaka_2018_wgs/sumstats/merged.peaks.eu.top.50k.bed", sep="\t", header=None)
#print(peaks_data_merged.head())
beds_var = pybedtools.BedTool.from_dataframe(peaks_var)

dataf1 = pybedtools.BedTool.from_dataframe(dataf1)
peak_bed = pybedtools.BedTool.from_dataframe(peaks_data_merged) # this is a merged peaks file
peaks_invar = peak_bed.intersect(beds_var, c=True)

In [15]:
peaks_invar = peaks_invar.to_dataframe()

In [19]:
peaks_invar = peaks_invar[peaks_invar["name"]>0]

In [23]:
datafn["pos0"] = datafn["var.pos_hg38"] - 1
all_var = datafn[["var.chr", "pos0", "var.pos_hg38"]].reset_index(drop=True)
beds_var = pybedtools.BedTool.from_dataframe(all_var)
peaks_invar = pybedtools.BedTool.from_dataframe(peaks_invar)
var_inpeaks = beds_var.intersect(peaks_invar, c=True)

In [25]:
var_inpeaks = var_inpeaks.to_dataframe()

In [26]:
var_inpeaks.shape

(130268, 4)

In [28]:
datafn["sigpeaks"] = var_inpeaks["name"]>0

In [30]:
datafn.head(1)

,var.chr,var.pos_hg38,var.allele1,var.allele2,obs.beta,obs.pval,var.isused,pred.enformer.encsr000emt.variantscore.global_sad,pred.enformer.encsr000emt.variantscore.global_sar,pred.chrombpnet.encsr637xsc.variantscore.logfc,...,pred.chrombpnet.encsr637xsc_50m.variantscore.jsd,pred.chrombpnet.encsr637xsc_50m.variantscore.ips,pred.chrombpnet.encsr000emt.variantscore.logfc,pred.chrombpnet.encsr000emt.variantscore.jsd,pred.chrombpnet.encsr000emt.variantscore.ips,pred.enformer.encsr000emt.variantscore.local_logfc,var.rsid,log10p,pos0,sigpeaks
0,chr1,29368,G,A,-0.555243,0.1623,False,NaN,NaN,0.01633,...,0.015501,0.000639,-0.022719,0.014977,0.000281,-0.0,chr1_29368_G_A,0.789681,29367,False


In [34]:
datafn[(datafn["log10p"]<3) & (datafn["sigpeaks"])].shape

(3157, 23)

In [37]:
sum(datafn[(datafn["log10p"]<3) & (datafn["sigpeaks"])]["var.isused"])

2946

In [40]:
datafn[(datafn["log10p"]<3) & (datafn["sigpeaks"])]["var.isused"]

20        False
21        False
22        False
23         True
113       False
          ...  
107204     True
107210     True
107218     True
107250     True
107251     True
Name: var.isused, Length: 3157, dtype: bool

In [46]:
datafn.loc[(datafn["log10p"]<3) & (datafn["sigpeaks"]),"var.isused"] = False

In [47]:
datafn[(datafn["log10p"]<3) & (datafn["sigpeaks"])]["var.isused"]

20        False
21        False
22        False
23        False
113       False
          ...  
107204    False
107210    False
107218    False
107250    False
107251    False
Name: var.isused, Length: 3157, dtype: bool

In [48]:
datafn.head(1)

,var.chr,var.pos_hg38,var.allele1,var.allele2,obs.beta,obs.pval,var.isused,pred.enformer.encsr000emt.variantscore.global_sad,pred.enformer.encsr000emt.variantscore.global_sar,pred.chrombpnet.encsr637xsc.variantscore.logfc,...,pred.chrombpnet.encsr637xsc_50m.variantscore.jsd,pred.chrombpnet.encsr637xsc_50m.variantscore.ips,pred.chrombpnet.encsr000emt.variantscore.logfc,pred.chrombpnet.encsr000emt.variantscore.jsd,pred.chrombpnet.encsr000emt.variantscore.ips,pred.enformer.encsr000emt.variantscore.local_logfc,var.rsid,log10p,pos0,sigpeaks
0,chr1,29368,G,A,-0.555243,0.1623,False,NaN,NaN,0.01633,...,0.015501,0.000639,-0.022719,0.014977,0.000281,-0.0,chr1_29368_G_A,0.789681,29367,False


In [49]:
datafn.shape

(130268, 23)

In [50]:
datafn.to_csv("full_variants/caqtls.eu.lcls.benchmarking.all.addfilter.tsv", sep="\t", header=True, index=False)